# World of Quantum Hands-on: QisK-I-T

This Notebook is divided into 2 parts:
- Code for error suppresion & dynamic decopuling Research Question (RQ1)
- Code for automated supervised learning best configuration Prediction in Optimization stage (RQ2)

    - Data-Preparation & Pass Selection and Experimentation
    - Model-Training and Experimentation
    - Evaluation

## Technical: Before you begin
- `conda create -n [env_name]`
- `conda activate [env_name]`
- `pip install -r ./requirements.txt`
- `conda update -all`

## Imports

In [83]:
import ast
import numpy as np
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, hamming_loss
from sklearn.metrics import f1_score
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import joblib
from pathlib import Path
from datetime import datetime
from scipy.stats import skew, kurtosis



# 1. Code for error suppresion & dynamic decopuling Research Question (RQ1)

In [84]:
# PLACEHOLDER RINORS AND JOSHUAS CODE

# 2. Code for automated supervised learning best configuration Prediction in Optimization stage (RQ2)

Below is the visualized pipeline

![My diagram](Resources/graph.png)

## Data-Preparation & Pass Selection and Experimentation

### hier der hackaton.py etc.

## Model-Training and Experimentation

### Data preprocessing

We do following steps:

1. a
2. b
3. c

In [85]:
dataset = pd.read_csv('data/training_data.csv', header=None)

dataset.rename(
    columns={
      dataset.columns[0]:  "ID",
      dataset.columns[-1]: "configuration"
    },
    inplace=True
)

# here we keep only the top 3 results for each circuit. After a shallow analysis of the dataset, the best improvements are seen in the top 3 results and after that the proposed configurations are converging to the results seen from before the optimization step. Other possible solution would be to add more weighting to the top 3 instances and keeping the other 7, but this is something we will explore in the future.


dataset = (
    dataset
    .groupby("ID", sort=False)
    .head(3)
    .reset_index(drop=True)
)

dataset.head()

,ID,1,2,3,4,5,6,7,8,configuration
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,0,110,0,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,0,110,0,"[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [86]:
df_configurations_with_id_with_duplicates = dataset[['ID', 'configuration']].copy()
df_features_with_id = dataset.drop(['configuration'], axis=1).copy()

In [87]:
df_configurations_with_id_with_duplicates.head()

,ID,configuration
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_11.qasm,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_11.qasm,"[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [88]:
df_features_with_id.head()

,ID,1,2,3,4,5,6,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
3,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,0,110,0
4,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,0,110,0


In [89]:
df_configurations_with_id_with_duplicates['config_arr'] = (
    df_configurations_with_id_with_duplicates['configuration']
      .apply(ast.literal_eval)          # "[1,0,1,...]" → [1,0,1,...]
      #.apply(lambda lst: np.array(lst, dtype=int))
)

# here we explored the option of aggregating the top 10 results of each instance by applying pointwise "or" operation, but this diluted the distance from optimal solution even further and gave unreliable result. This was expected, as top results configuration can be different to the further ones, and the or operation, in some cases, just made the configuration vector apply near all optimizations, bumping the runtime and dilluting the results, as some of the passes may cancel each other out.

#df_ored_configuration = (
#    df_configurations_with_id_with_duplicates
#      .groupby('ID')['config_arr']
#      .agg(lambda arrs: np.bitwise_or.reduce(arrs.tolist()))
#      .reset_index()
#      .rename(columns={'config_arr':'label_vec'})
#)

df_configurations = df_configurations_with_id_with_duplicates[['ID','config_arr']].copy()

In [90]:
df_configurations.head()

,ID,config_arr
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_11.qasm,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_11.qasm,"[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [91]:
df_features_with_id.head()

,ID,1,2,3,4,5,6,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
3,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,0,110,0
4,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,0,110,0


In [92]:
single_value_cols = df_features_with_id.columns[df_features_with_id.nunique(dropna=False) == 1]
single_value_cols

Index([6], dtype='object')

In [93]:
df_feat = df_features_with_id.drop(single_value_cols, axis=1).copy()
df_feat.head()

,ID,1,2,3,4,5,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
3,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,110,0
4,ae_nativegates_ibm_qiskit_opt0_11.qasm,11,204,22,441,110,110,0


In [94]:
df_feat.drop(['ID'], inplace=True, axis=1)
df_configurations.drop(['ID'], inplace=True, axis=1)

In [95]:
df_feat

,1,2,3,4,5,7,8
0,10,183,20,375,90,90,0
1,10,183,20,375,90,90,0
2,10,183,20,375,90,90,0
3,11,204,22,441,110,110,0
4,11,204,22,441,110,110,0
...,...,...,...,...,...,...,...
646,8,50,16,252,84,84,0
647,8,50,16,252,84,84,0
648,9,54,18,297,108,108,0
649,9,54,18,297,108,108,0


In [96]:
df_configurations

,config_arr
0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...
646,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
647,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
648,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
649,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [97]:
df_feat.dtypes

1    int64
2    int64
3    int64
4    int64
5    int64
7    int64
8    int64
dtype: object

In [98]:
stats = pd.DataFrame({
    'mean':        df_feat.mean(),
    'std':         df_feat.std(),
    'range':       df_feat.max() - df_feat.min(),
    'IQR':         df_feat.quantile(0.75) - df_feat.quantile(0.25),
    'CV':          df_feat.std() / df_feat.mean().abs().replace(0, np.nan),
    'skewness':    df_feat.apply(skew, nan_policy='omit'),
    'kurtosis':    df_feat.apply(lambda x: kurtosis(x, nan_policy='omit')),
    'outlier_%':   df_feat.apply(lambda x: ((x < x.mean() - 3*x.std()) | (x > x.mean() + 3*x.std())).mean() * 100)
})

# sorting by biggest disparities.
stats_sorted = stats.sort_values(by='std', ascending=False)
print(stats_sorted)


         mean          std  range    IQR        CV   skewness    kurtosis  \
4  948.654378  2687.351763  38101  910.0  2.832804  12.269458  166.051440   
2  266.036866  1295.022897  18741  169.0  4.867832  13.505387  188.860029   
5  336.400922  1026.233478  14532  387.0  3.050626  12.279628  166.321343   
7  336.447005  1026.218482  14531  387.0  3.050164  12.280046  166.329026   
3   28.041475    16.656340     77   24.0  0.593989   0.680163    0.261058   
1   13.976959     8.314089     38   12.0  0.594843   0.715896    0.293709   
8    0.046083     0.209826      1    0.0  4.553224   4.329932   16.748309   

   outlier_%  
4   0.460829  
2   0.460829  
5   0.460829  
7   0.460829  
3   0.460829  
1   0.921659  
8   4.608295  


The output from above can be analyzed in the following way:
- skewness measures the symmetry of distribution, with 0 symbolizing perfectly symmetric. Values of 12 - 13 are really high, meaning we have a few very large values outliers.
- mean and std are also very high (or interquartile range). This means some features possibly completly overshadow others.
- outlier_% computes for each feature column the mean and deviation and marks those which lie outside of 3 standard deviations. This flags extreme points. Features 1 and even more so feature 8 have high outlier percentage. Value of 4.61 % means on average one in twenty rows is beyond 3 standard deviations, meaning this explanatory variable has a very heavy high values outliers (is heavy right tailed).
- CV is the standard deviation / mean. The bigger it is, the bigger is the noise, meaning intuitevly for instance a lot of rows with value 0 and then a few with value 1000 -> chaotic, unstable scale of the feature.
- Kurtosis measures how much propability lives in the tails vs. shoulders of distribution. In other words, "normal" kurtosis of about 3 represents classic bell kurve - tails are moderate and outliers are rare but possible. In the example above, for instance feature 4 has kurtosis of 166. This means outliers are not occasional, but they are the "default" baked in the data. (Here the word "outlier" is now quite counter-intuitiv).

This could possibly explain, as of the first prototype, the very low value of macro-F1 of 0.26. Extreme feature values could correspond to rare passes, so the labels, or passes that are in the minority are completly overwhelmed by two things: class imbalance (difference in size between how some columns / explanatory variables overwhelming others) and by feature scale imbalance (meaning for instance a lot of 0's in one column and then minority of very high values, e.g. [0,0,0, ..., 1243, 0, ..., 0])


Solution: at first I have tried the scaling, but as the name suggests in only rescaled the features but does not fix the skewness or outliers. I will now look into PowerTransformer, QuantileTransformer, RobustScaler, clipping the extreme values before scaling...

In [99]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_feat)
df_scaled = pd.DataFrame(X_scaled, columns=df_feat.columns, index=df_feat.index)
df_scaled

,1,2,3,4,5,7,8
0,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
1,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
2,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
3,-0.358337,-0.047941,-0.362992,-0.189050,-0.220783,-0.220831,-0.219793
4,-0.358337,-0.047941,-0.362992,-0.189050,-0.220783,-0.220831,-0.219793
...,...,...,...,...,...,...,...
646,-0.719448,-0.166949,-0.723492,-0.259434,-0.246138,-0.246186,-0.219793
647,-0.719448,-0.166949,-0.723492,-0.259434,-0.246138,-0.246186,-0.219793
648,-0.599078,-0.163858,-0.603326,-0.242676,-0.222733,-0.222782,-0.219793
649,-0.599078,-0.163858,-0.603326,-0.242676,-0.222733,-0.222782,-0.219793


In [100]:
y = np.vstack(df_configurations['config_arr'].values)
X = df_scaled.values

In [101]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=43
)

### Configuration for cross-validation of hyperparameters and of the model to gain reliable insights

In [102]:
# Configuration
DO_HPO          = True # hyperparameter search
HPO_NUM_TRIALS  = 60
N_OUTER_FOLDS   = 10
N_INNER_FOLDS   = 5
RANDOM_STATE    = 43

BASELINE_PARAMS = {
    "n_estimators": 1000,
}

In [103]:
outer_cv = MultilabelStratifiedKFold(
    n_splits=N_OUTER_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

inner_cv = MultilabelStratifiedKFold(
    n_splits=N_INNER_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

In [104]:
if DO_HPO:
    def objective(trial):
        param = {
            "n_estimators":       trial.suggest_int("n_estimators", 50, 1000),
            "max_depth":          trial.suggest_int("max_depth", 3, 100),
            "learning_rate":      trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "subsample":          trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree":   trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "objective":          "binary:logistic",
            "tree_method":        "hist",
            "base_score":          0.5,
            "eval_metric":        "logloss",
            "random_state":       RANDOM_STATE
        }

        classifier = XGBClassifier(
        **param,
        )

        multiLabelClassifier = MultiOutputClassifier(classifier, n_jobs=-1)

        scores = cross_val_score(
            multiLabelClassifier,
            X_trainval,
            y_trainval,
            cv=inner_cv,
            scoring=make_scorer(f1_score, average='micro', zero_division=0),
            n_jobs=-1
        )
        # return the mean macro-F1 across folds
        return scores.mean()

    # 4) create & run the study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=HPO_NUM_TRIALS)

    # 5) extract best params
    best_params = study.best_params
    print("Best hyperparameters:", best_params)
else:
    best_params = BASELINE_PARAMS.copy()

[I 2025-06-27 10:24:54,786] A new study created in memory with name: no-name-626c5db7-122f-4b4c-aa54-5d946e6ba188
[I 2025-06-27 10:24:57,041] Trial 0 finished with value: 0.6173882578932277 and parameters: {'n_estimators': 633, 'max_depth': 26, 'learning_rate': 0.0509378605786282, 'subsample': 0.8186801340503982, 'colsample_bytree': 0.9662968292910157}. Best is trial 0 with value: 0.6173882578932277.
[I 2025-06-27 10:24:59,151] Trial 1 finished with value: 0.6257206852262425 and parameters: {'n_estimators': 919, 'max_depth': 13, 'learning_rate': 0.015646812721410035, 'subsample': 0.9619638780108596, 'colsample_bytree': 0.900575851602309}. Best is trial 1 with value: 0.6257206852262425.
[I 2025-06-27 10:25:01,171] Trial 2 finished with value: 0.6269769743580158 and parameters: {'n_estimators': 610, 'max_depth': 16, 'learning_rate': 0.004154934072331043, 'subsample': 0.7575304494199881, 'colsample_bytree': 0.7437348008016023}. Best is trial 2 with value: 0.6269769743580158.
[I 2025-06-27

Best hyperparameters: {'n_estimators': 398, 'max_depth': 66, 'learning_rate': 0.0010156063537942682, 'subsample': 0.9299410903626396, 'colsample_bytree': 0.7648857188331771}


In [105]:
base_clf = XGBClassifier(
    **best_params,
    objective='binary:logistic',
    random_state=RANDOM_STATE,
    tree_method='hist',
    base_score=0.5,
    eval_metric='logloss'
)

multiLabelClassifier = MultiOutputClassifier(base_clf, n_jobs=-1)
scoring = {
    "f1_micro": make_scorer(f1_score, average='micro', zero_division=0),
    "f1_macro": make_scorer(f1_score, average='macro', zero_division=0),
    "hamming": make_scorer(hamming_loss)
}

cv_results = cross_validate(
    multiLabelClassifier, X_trainval, y_trainval,
    cv=outer_cv,
    scoring=scoring,
    return_train_score=False,
    n_jobs=-1
)

print("Outer-fold scores:")
for name in scoring:
    mean = cv_results[f"test_{name}"].mean()
    std  = cv_results[f"test_{name}"].std()
    print(f"  {name:>10s}: {mean:7.3f} ± {std:.3f}")

Outer-fold scores:
    f1_micro:   0.642 ± 0.023
    f1_macro:   0.218 ± 0.006
     hamming:   0.091 ± 0.006


## Evaluation Metrics

### F1-micro

* **Measures:** Overall accuracy by pooling all label decisions.
* **Formula:**

  $$
  \frac{2 \cdot \text{TP}_{\text{total}}}
       {2 \cdot \text{TP}_{\text{total}} + \text{FP}_{\text{total}} + \text{FN}_{\text{total}}}
  $$

---

### F1-macro

* **Measures:** Average F1 across labels, treating each label equally.
* **Formula:**

  $$
  \frac{1}{L}\sum_{i=1}^{L}
  \frac{2 \cdot \text{TP}_i}
       {2 \cdot \text{TP}_i + \text{FP}_i + \text{FN}_i}
  $$

---

### Hamming Loss

* **Measures:** Fraction of incorrect label decisions.
* **Formula:**

  $$
  \frac{\text{FP}_{\text{total}} + \text{FN}_{\text{total}}}
       {(\text{number of labels}) \times (\text{number of samples})}
  $$

Interpretation of final test-set metrics:
 F1-micro: 0.691
 F1-macro: 0.260
 Hamming : 0.075

- Micro-F1 & Hamming: We are getting ~68 % of bits right and misclassifying only 7–8 % on average
- Macro-F1: 0.26 is low, indicating we struggle on rare passes. The reason is possibly unsufficient data preprocessing, meaning underrepresenting rare instances, due to the limited time. This could be possibly gratly improved by better preprocessing / data weighting with custom weight functions.


In [106]:
multiLabelClassifier.fit(X_trainval, y_trainval)
y_pred = multiLabelClassifier.predict(X_test)

# Final test-set metrics, with zero_division=0 and formatted
f1_micro_test = f1_score(y_test, y_pred, average='micro', zero_division=0)
f1_macro_test = f1_score(y_test, y_pred, average='macro', zero_division=0)
hamming_test  = hamming_loss(y_test, y_pred)

print("\nFinal test-set metrics:")
print(f" F1-micro: {f1_micro_test:.3f}")
print(f" F1-macro: {f1_macro_test:.3f}")
print(f" Hamming : {hamming_test:.3f}")


Final test-set metrics:
 F1-micro: 0.654
 F1-macro: 0.226
 Hamming : 0.087


In [107]:
# number of samples you want to inspect
n_samples = 10

# pick n_samples random indices from the test set
rand_idx = np.random.choice(len(y_test), size=n_samples, replace=False)

# build a DataFrame showing y_test vs y_pred for those indices
df_compare = pd.DataFrame({
    "True configuration": [y_test[i].tolist() for i in rand_idx],
    "Predicted configuration": [y_pred[i].tolist() for i in rand_idx],
}, index=rand_idx)

display(df_compare)

,True configuration,Predicted configuration
120,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
101,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
52,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
113,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
82,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
79,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
83,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
34,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
33,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."


In [108]:
multiLabelClassifier.fit(X, y)

,estimator,"XGBClassifier...ree=None, ...)"
,n_jobs,-1
,objective,'binary:logistic'
,base_score,0.5
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7648857188331771
,device,None
,early_stopping_rounds,None


In [109]:
data_dir = Path("models")
data_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

model_filename  = f"qiskit_pass_predictor_{timestamp}.joblib"

model_path  = data_dir / model_filename


joblib.dump(multiLabelClassifier, model_path)
print(f"Model saved to {model_path}")

Model saved to models/qiskit_pass_predictor_20250627_102555.joblib


## Evaluation
We use the trained model to predict the optimal configuration for new circuits.
Then, we evaluate performance based on the number of qubits in the transpiled output, comparing it against Qiskit's default PassManager strategies: fast, normal, and slow.

In [110]:
# TODO: Add evaluation code against default optimization routine